In [1]:
import sys
sys.path.append("/pod/2/ke-lab/LUOZ/Singularity/iM6A")

In [2]:
from keras.models import load_model
from pkg_resources import resource_filename
import numpy as np
import pandas as pd
from Bio.Seq import Seq
import keras.backend as kb

Using TensorFlow backend.


In [3]:
def one_hot_encode(seq):

    map = np.asarray([[0, 0, 0, 0],
                      [1, 0, 0, 0],
                      [0, 1, 0, 0],
                      [0, 0, 1, 0],
                      [0, 0, 0, 1]])

    seq = seq.upper().replace('A', '\x01').replace('C', '\x02')
    seq = seq.replace('G', '\x03').replace('T', '\x04').replace('N', '\x00')

    return map[np.fromstring(seq, np.int8) % 5]

In [4]:
def categorical_crossentropy_2d(y_true, y_pred):
    # Standard categorical cross entropy for sequence outputs

    return - kb.mean(y_true[:, :, 0]*kb.log(y_pred[:, :, 0]+1e-10)
                   + y_true[:, :, 1]*kb.log(y_pred[:, :, 1]+1e-10))

In [5]:
context = 10000

In [6]:
paths = ('Test/humanRAC10000_c{}.h5'.format(x) for x in range(1, 6))

In [7]:
models = [load_model(resource_filename('m6AAI', x), custom_objects={'categorical_crossentropy_2d': categorical_crossentropy_2d}) for x in paths]

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


### Read data

In [8]:
Fasta = pd.read_csv("/pod/2/ke-lab/LUOZ/m6AQTL/Fasta_m6A_QTL_SigSite.csv", keep_default_na=False)

In [9]:
Fasta

,chrom,SNPID,strand,PeakStart,PeakEnd,name,Peak,POS,REF,ALT,txStart,txEnd,cdsStart,cdsEnd,LastExonStart,LastExonEnd,Sequence,GeneLength,SNPposition
0,chr20,rs3182973,+,57569747,57570139,NELFCD,chr20:57569747-57570139_NELFCD_+,57569938,A,G,57556262,57570188,57556310,57569731,57569696,57570188,gaggcgcgcggccgcgcgtgcgcccgcccgcccgtccccgcctcgc...,13926,13676
1,chr20,rs7397,+,61431004,61431648,MRGBP,chr20:61431004-61431648_MRGBP_+,61431557,A,C,61427804,61431945,61427875,61430995,61430807,61431945,AGTGCGCCTGCGCGGAGCTCGTGGCCGCGCCTGCTCCCGCCGGGGG...,4141,3753
2,chr20,rs8567,+,62522205,62522403,TPD52L2,chr20:62522205-62522403_TPD52L2_+,62522315,A,G,62496642,62522898,62496718,62521298,62521202,62522898,ACGCGGCGAGCTTCTCCCGGCGCCGCCCGCTCGGCTCCCATAGCGC...,26256,25673
3,chr20,rs4810237,+,37434398,37518294,PPP1R16B,chr20:37434398-37518294_PPP1R16B_+,37487150,G,T,37434347,37551667,37464568,37547309,37546799,37551667,CTCTGTCGCCGCTGGAGCAGCTGCGGCTCGGGGACCCACAGACACA...,117320,52803
4,chr20,rs73112067,+,37434398,37518294,PPP1R16B,chr20:37434398-37518294_PPP1R16B_+,37437740,G,A,37434347,37551667,37464568,37547309,37546799,37551667,CTCTGTCGCCGCTGGAGCAGCTGCGGCTCGGGGACCCACAGACACA...,117320,3393
5,chr20,rs6043684,+,15967385,16030520,MACROD2,chr20:15967385-16030520_MACROD2_+,16023836,C,A,13976014,16033842,13976409,16030521,16030474,16033842,GCGTGCGCCCTGGGGCCCAAGTTGGGGCGCGCCGTGGCTAGAACAC...,2057828,2047822
6,chr20,rs4811239,+,36488704,36500482,CTNNBL1,chr20:36488704-36500482_CTNNBL1_+,36492910,A,G,36322407,36500531,36322524,36500415,36500326,36500531,ATTGGCTCTGCGGCTCCGCTCGCCGCACTTTACGGCAGTGTGGCTG...,178124,170503
7,chr20,rs60453471,+,36355753,36365885,CTNNBL1,chr20:36355753-36365885_CTNNBL1_+,36356630,T,C,36322407,36500531,36322524,36500415,36500326,36500531,ATTGGCTCTGCGGCTCCGCTCGCCGCACTTTACGGCAGTGTGGCTG...,178124,34223
8,chr20,rs736953,+,32399110,32436316,CHMP4B,chr20:32399110-32436316_CHMP4B_+,32415089,G,T,32399109,32442172,32399274,32441366,32441301,32442172,ggtggcgcggccgggcccagctgcggggcggaggcggaggcggagg...,43063,15980
9,chr20,rs2092737,+,32399110,32436316,CHMP4B,chr20:32399110-32436316_CHMP4B_+,32428135,A,C,32399109,32442172,32399274,32441366,32441301,32442172,ggtggcgcggccgggcccagctgcggggcggaggcggaggcggagg...,43063,29026


### Select positive strand

In [10]:
Positive = Fasta[Fasta["strand"]=="+"]
Positive = Positive.reset_index(drop = True)

In [11]:
Positive

,chrom,SNPID,strand,PeakStart,PeakEnd,name,Peak,POS,REF,ALT,txStart,txEnd,cdsStart,cdsEnd,LastExonStart,LastExonEnd,Sequence,GeneLength,SNPposition
0,chr20,rs3182973,+,57569747,57570139,NELFCD,chr20:57569747-57570139_NELFCD_+,57569938,A,G,57556262,57570188,57556310,57569731,57569696,57570188,gaggcgcgcggccgcgcgtgcgcccgcccgcccgtccccgcctcgc...,13926,13676
1,chr20,rs7397,+,61431004,61431648,MRGBP,chr20:61431004-61431648_MRGBP_+,61431557,A,C,61427804,61431945,61427875,61430995,61430807,61431945,AGTGCGCCTGCGCGGAGCTCGTGGCCGCGCCTGCTCCCGCCGGGGG...,4141,3753
2,chr20,rs8567,+,62522205,62522403,TPD52L2,chr20:62522205-62522403_TPD52L2_+,62522315,A,G,62496642,62522898,62496718,62521298,62521202,62522898,ACGCGGCGAGCTTCTCCCGGCGCCGCCCGCTCGGCTCCCATAGCGC...,26256,25673
3,chr20,rs4810237,+,37434398,37518294,PPP1R16B,chr20:37434398-37518294_PPP1R16B_+,37487150,G,T,37434347,37551667,37464568,37547309,37546799,37551667,CTCTGTCGCCGCTGGAGCAGCTGCGGCTCGGGGACCCACAGACACA...,117320,52803
4,chr20,rs73112067,+,37434398,37518294,PPP1R16B,chr20:37434398-37518294_PPP1R16B_+,37437740,G,A,37434347,37551667,37464568,37547309,37546799,37551667,CTCTGTCGCCGCTGGAGCAGCTGCGGCTCGGGGACCCACAGACACA...,117320,3393
5,chr20,rs6043684,+,15967385,16030520,MACROD2,chr20:15967385-16030520_MACROD2_+,16023836,C,A,13976014,16033842,13976409,16030521,16030474,16033842,GCGTGCGCCCTGGGGCCCAAGTTGGGGCGCGCCGTGGCTAGAACAC...,2057828,2047822
6,chr20,rs4811239,+,36488704,36500482,CTNNBL1,chr20:36488704-36500482_CTNNBL1_+,36492910,A,G,36322407,36500531,36322524,36500415,36500326,36500531,ATTGGCTCTGCGGCTCCGCTCGCCGCACTTTACGGCAGTGTGGCTG...,178124,170503
7,chr20,rs60453471,+,36355753,36365885,CTNNBL1,chr20:36355753-36365885_CTNNBL1_+,36356630,T,C,36322407,36500531,36322524,36500415,36500326,36500531,ATTGGCTCTGCGGCTCCGCTCGCCGCACTTTACGGCAGTGTGGCTG...,178124,34223
8,chr20,rs736953,+,32399110,32436316,CHMP4B,chr20:32399110-32436316_CHMP4B_+,32415089,G,T,32399109,32442172,32399274,32441366,32441301,32442172,ggtggcgcggccgggcccagctgcggggcggaggcggaggcggagg...,43063,15980
9,chr20,rs2092737,+,32399110,32436316,CHMP4B,chr20:32399110-32436316_CHMP4B_+,32428135,A,C,32399109,32442172,32399274,32441366,32441301,32442172,ggtggcgcggccgggcccagctgcggggcggaggcggaggcggagg...,43063,29026


In [12]:
Sequence = Positive["Sequence"].tolist()
Ref = Positive["REF"].tolist()
Alt = Positive["ALT"].tolist()
Position = Positive["SNPposition"].tolist()

In [13]:
for i in range(len(Sequence)):
    input_sequence = Sequence[i]
    seq = input_sequence[(Positive.loc[i,("SNPposition")]-1):(Positive.loc[i,("SNPposition")])]
    print(seq)

A
A
A
G
G
c
A
t
g
A
C
c
G
g
A
g
C
G
A
T
G
G
c
A
a
C
g
c
C
g
C
G
a
g
a
t
G
T
c
a
g
c
A
c
C
G
A
t
T
G
g
a
g
A
g
c
C
C
C
T
c
c
g
G
a
T
a
C
G
g
C
G
t
a
T
T
A
A
G
g
t
t
T
a
g
c
G
t
c
g
t
a
t
g
g
c
t
t
g
g
a
C
C
a
a
c
c
a
a
G
a
G
g
g
C
T
T
A
C
A
g
G
T
T
T
a
C
c
C
C
G
A
c
A
T
G
a
A
A
G
G
t
G
G
A
A
A
g
g
G
a
c
c
A
A
T
A
A
G
G
T
C
G
A
C
G
A
c
t
T
t
T
c
C
C
a
c
g
c
C
g
g
A
G
G
A
G
c
t
t
c
a
T
G
A
T
a
a
a
A
G
G
C
C
C
A
G
G
T
A
a
g
c
c
c
c
a
T
g
c
T
T
C
T
C
c
A
t
g
g
g
A
T
a
G
G
A
T
G
A
G
A
c
A
a
a
G
a
C
T
a
c
g
T
T
A
c
A
t
g
t
a
a
T
a
a
c
t
T
T
a
C
C
A
C
t
t
T
C
a
G
T
A
T
c
g
g
C
G
a
C
g
T
c
c
G
A
C
C
G
G
G
G
G
G
G
T
T
C
C
C
G
C
c
T
G
G
T
g
G
c
A
T
t
G
t
g
G
c
a
T
G
t
g
T
t
C
c
c
C
C
C
A
G
t
t
T
a
G
C
T
a
G
A
G
G
G
a
G
C
G
G
G
a
a
C
C
A
C
a
a
t
g
A
a
G
a
T
T
A
T
T
T
T
c
T
G
C
G
t
A
G
C
T
G
G
g
c
G
A
T
G
G
g
G
G
G
C
C
t
c
t
A
c
T
g
c
c
a
g
G
c
g
a
T
t
C
T
T
a
A
G
C
c
c
g
G
C
g
c
c
A
G
a
A
G
C
A
C
G
T
A
C
G
C
T
T
T
C
T
A
C
C
T
G
A
G
T
T
T
c
T
g
t
a
t
T
g
a
C
T
c
G
C
A
g
c
g
C
A
T
a
g
C
C
A
t
G
C
T


In [14]:
for i in range(len(Sequence)):
    seq = Sequence[i]
    input_sequence = seq[:(Positive.loc[i,("SNPposition")]-1)] + Alt[i] + seq[(Positive.loc[i,("SNPposition")]):]
    
    x = one_hot_encode('N'*(context//2) + input_sequence + 'N'*(context//2))[None, :]
    y = np.mean([models[m].predict(x) for m in range(5)], axis=0)
    m6AAI_prob = y[0, :, 1]
    Probability = pd.DataFrame({'Probability':m6AAI_prob})
    
    df = pd.DataFrame(np.random.randn((Positive.loc[i,"GeneLength"]), 3))
    df.columns = ["name", "chrom", "strand"]
    df["name"] = Positive.loc[i,"SNPID"]
    df["chrom"] = Positive.loc[i,"chrom"]
    df["strand"] = Positive.loc[i,"strand"]
    
    list = range(Positive.loc[i,"txStart"], Positive.loc[i,"txEnd"])
    Start = pd.DataFrame(list, columns=["Start"])
    df = pd.concat([df, Start], axis=1)
    df["End"] = df["Start"]
    df = pd.concat([df, Probability], axis=1)
    # df = df.tail(Positive.loc[i,"LastExonLength"])       
    df.columns = ["name", "chrom", "strand", "Start", "End", "Probabilty"]
    df.to_csv("./m6AQTL/QTL/{}.bed".format(Positive.loc[i,"SNPID"]), sep="\t", index=False)


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  if sys.path[0] == '':


### Select negative sample

In [15]:
Negative = Fasta[Fasta["strand"]=="-"]
Negative = Negative.reset_index(drop = True)

In [16]:
Negative

,chrom,SNPID,strand,PeakStart,PeakEnd,name,Peak,POS,REF,ALT,txStart,txEnd,cdsStart,cdsEnd,LastExonStart,LastExonEnd,Sequence,GeneLength,SNPposition
0,chr20,rs238187,-,47887351,47892424,ZNFX1,chr20:47887351-47892424_ZNFX1_-,47890342,T,C,47862436,47894756,47863803,47892376,47862436,47866248,TTTTCTCAATCATTGTTTTTAATTGGCTTTATAAGCTAAAGTGCAT...,32320,4415
1,chr20,rs238188,-,47887351,47892424,ZNFX1,chr20:47887351-47892424_ZNFX1_-,47891332,G,C,47862436,47894756,47863803,47892376,47862436,47866248,TTTTCTCAATCATTGTTTTTAATTGGCTTTATAAGCTAAAGTGCAT...,32320,3425
2,chr20,rs238189,-,47887351,47892424,ZNFX1,chr20:47887351-47892424_ZNFX1_-,47891364,G,T,47862436,47894756,47863803,47892376,47862436,47866248,TTTTCTCAATCATTGTTTTTAATTGGCTTTATAAGCTAAAGTGCAT...,32320,3393
3,chr20,rs238190,-,47887351,47892424,ZNFX1,chr20:47887351-47892424_ZNFX1_-,47891646,C,T,47862436,47894756,47863803,47892376,47862436,47866248,TTTTCTCAATCATTGTTTTTAATTGGCTTTATAAGCTAAAGTGCAT...,32320,3111
4,chr20,rs238191,-,47887351,47892424,ZNFX1,chr20:47887351-47892424_ZNFX1_-,47892270,T,C,47862436,47894756,47863803,47892376,47862436,47866248,TTTTCTCAATCATTGTTTTTAATTGGCTTTATAAGCTAAAGTGCAT...,32320,2487
5,chr20,rs238208,-,47865128,47865525,ZNFX1,chr20:47865128-47865525_ZNFX1_-,47865372,G,A,47862436,47894756,47863803,47892376,47862436,47866248,TTTTCTCAATCATTGTTTTTAATTGGCTTTATAAGCTAAAGTGCAT...,32320,29385
6,chr20,rs236176,-,5922662,5923402,TRMT6,chr20:5922662-5923402_TRMT6_-,5922821,T,C,5918478,5931182,5919180,5931051,5918478,5919372,GGGAAAAGACATTTGTTTCCTTTAATTGAATAGTACACAAAAATTC...,12704,8362
7,chr20,rs394940,-,5922662,5923402,TRMT6,chr20:5922662-5923402_TRMT6_-,5923203,T,C,5918478,5931182,5919180,5931051,5918478,5919372,GGGAAAAGACATTTGTTTCCTTTAATTGAATAGTACACAAAAATTC...,12704,7980
8,chr20,rs1058007,-,7962076,7962324,TMX4,chr20:7962076-7962324_TMX4_-,7962128,C,T,7957994,8000476,7962897,8000260,7957994,7963268,GAGGATGGACACGAAGGaaaattttttttaacaaattaatattttt...,42482,38349
9,chr20,rs2076015,-,7962871,7963219,TMX4,chr20:7962871-7963219_TMX4_-,7963041,C,T,7957994,8000476,7962897,8000260,7957994,7963268,GAGGATGGACACGAAGGaaaattttttttaacaaattaatattttt...,42482,37436


In [17]:
Sequence = Negative["Sequence"].tolist()
Ref = Negative["REF"].tolist()
Alt = Negative["ALT"].tolist()
Position = Negative["SNPposition"].tolist()

In [18]:
for i in range(len(Sequence)):
    input_sequence = Sequence[i][::-1]
    input_sequence = Seq(input_sequence).complement()
    input_sequence = str(input_sequence)    
    seq = input_sequence[(Negative.loc[i,("SNPposition")]-1):(Negative.loc[i,("SNPposition")])]
    print(seq)

a
c
c
g
A
C
A
A
G
G
C
A
C
A
G
G
G
G
C
C
G
T
G
C
G
T
G
A
A
C
A
G
a
C
c
a
g
G
T
C
A
G
G
t
C
c
c
t
G
g
T
c
g
G
A
A
C
t
C
a
a
T
C
T
G
G
G
a
A
A
T
C
A
A
T
A
G
G
A
G
t
A
A
C
c
T
T
G
C
A
t
G
G
C
A
T
G
G
G
G
A
T
A
G
A
T
C
T
C
G
C
G
T
T
T
A
T
C
G
G
a
G
A
G
A
T
G
G
c
T
G
A
T
T
c
t
g
g
A
a
c
G
C
A
c
c
T
c
c
C
C
g
A
G
A
t
T
t
A
g
A
C
g
c
G
C
a
t
t
C
a
g
c
g
t
G
A
T
t
A
A
G
c
G
A
T
C
A
T
C
C
G
G
A
T
C
T
A
A
a
t
g
G
A
g
C
a
a
g
C
G
c
G
t
g
G
c
C
T
G
C
g
C
G
G
g
a
A
a
g
T
G
A
A
C
C
C
t
T
c
A
G
T
T
c
T
g
T
g
G
T
G
A
G
G
C
G
g
g
g
t
a
C
G
C
C
T
C
T
A
G
A
G
t
C
C
c
t
g
C
T
t
T
g
A
G
T
G
A
t
g
C
G
c
G
C
A
T
a
t
T
c
G
G
a
A
G
A
G
a
C
A
C
G
C
A
T
G
A
T
G
T
C
A
A
c
C
G
T
g
T
g
A
G
A
t
t
C
t
C
G
a
t
C
T
C
a
G
g
a
g
t
t
t
A
A
g
t
c
G
T
C
G
C
A
t
a
T
G
A
A
A
G
g
t
t
g
A
T
G
T
C
a
T
C
C
a
t
c
g
T
a
A
t
a
a
g
c
t
t
t
a
a
C
t
c
c
T
g
c
C
C
C
G
C
T
T
G
a
C
A
c
T
t
C
G
T
g
g
T
T
G
A
C
G
A
a
C
T
a
C
a
T
a
T
A
a
T
A
A
t
T
A
C
A
A
C
T
a
t
g
c
g
G
C
G
C
g
a
g
a
t
g
C
G
G
t
G
C
G
T
G
T
T
A
t
T
T
G
T
T
a
A
G
A
c
A
c
a
G


In [19]:
for i in range(len(Sequence)):
    seq = Sequence[i][::-1]
    seq = Seq(seq).complement()
    seq = str(seq)
    
    input_sequence = seq[:(Negative.loc[i,("SNPposition")]-1)] + str(Seq(Alt[i]).complement()) + seq[(Negative.loc[i,("SNPposition")]):]
    
    x = one_hot_encode('N'*(context//2) + input_sequence + 'N'*(context//2))[None, :]
    y = np.mean([models[m].predict(x) for m in range(5)], axis=0)
    m6AAI_prob = y[0, :, 1]
    Probability = pd.DataFrame({'Probability':m6AAI_prob})
    Probability.sort_index(inplace=True, ascending=False)
    Probability = Probability.reset_index(drop = True)
    
    df = pd.DataFrame(np.random.randn((Negative.loc[i,"GeneLength"]), 3))
    df.columns = ["name", "chrom", "strand"]
    df["name"] = Negative.loc[i,"SNPID"]
    df["chrom"] = Negative.loc[i,"chrom"]
    df["strand"] = Negative.loc[i,"strand"]
    
    list = range(Negative.loc[i,"txStart"], Negative.loc[i,"txEnd"])
    Start = pd.DataFrame(list, columns=["Start"])
    df = pd.concat([df, Start], axis=1)    
    df["End"] = df["Start"]
    df = pd.concat([df, Probability], axis=1)    
    # df = df.head(Negative.loc[i,"LastExonLength"])    
    df.columns = ["name", "chrom", "strand", "Start", "End", "Probabilty"]
    df.to_csv("./m6AQTL/QTL/{}.bed".format(Negative.loc[i,"SNPID"]), sep="\t", index=False)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  if sys.path[0] == '':
